# Survival analysis Plugins


**AutoPrognosis** provides a set of default predicton plugins and can be extended with any number of other plugins.

In this tutorial, we will focus on the __survival analysis__ tasks.

### Plugins 101

Every **AutoPrognosis plugin** must implement the **`Plugin`** interface provided by `autoprognosis/plugins/core/base_plugin.py`.

Each **AutoPrognosis prediction plugin** must implement the **`PredictionPlugin`** interface provided by `autoprognosis/plugins/prediction/base.py`

__Warning__ : If a plugin doesn't override all the abstract methods, it won't be loaded by the library.




__API__ : Every prediction plugin must implement the following methods:
- `name()` - a static method that returns the name of the plugin. e.g., neural_nets, perceptron, etc.

- `subtype()` - a static method that returns the plugin's subtype. e.g., "classification", "survival_analysis" etc. It will be used for filtering the plugin in the optimization process.
    
- `hyperparameter_space()` - a static method that returns the hyperparameters that can be tuned during the optimization. The method will return a list of `skopt.space.Dimension` derived objects.

### Setup

In [ ]:
# stdlib
import sys
import warnings

import numpy as np
import pandas as pd
import tabulate

# third party
from IPython.display import HTML, display

if not sys.warnoptions:
    warnings.simplefilter("ignore")

### Loading the RiskEstimation plugins

Make sure that you have installed AutoPrognosis in your workspace.

You can do that by running `pip install .` in the root of the repository.

In [ ]:
# autoprognosis absolute
from autoprognosis.plugins.prediction.risk_estimation import (
    RiskEstimation,
    RiskEstimationPlugin,
)

predictors = RiskEstimation()

### List the existing plugins

In [ ]:
predictors.list_available()

## Benchmarks

We test the prediction plugins using the [METABRIC dataset](https://github.com/jaredleekatzman/DeepSurv/tree/master/experiments/data).

### Loading the data

In [ ]:
# third party
import numpy as np
from pycox import datasets
from sklearn.model_selection import train_test_split

df = datasets.metabric.read_df()

X = df.drop(columns=["duration", "event"])
T = df["duration"]
E = df["event"]

X = X[T > 0]
E = E[T > 0]
T = T[T > 0]

time_horizons = np.linspace(T.min(), T.max(), 5)[1:-1]

X

### Duration benchmarks

__About__ : This step measures the fit_predict duration for each plugin on the dataset. The times are reported in milliseconds.

In [ ]:
# stdlib
import time

# third party
from tqdm import tqdm

duration = []

plugins = predictors.list_available()

for plugin in tqdm(plugins):
    plugin_duration = [plugin]
    ctx = predictors.get(plugin)

    start = time.time() * 1000
    ctx.fit(X, T, E)
    ctx.predict(X, time_horizons)

    plugin_duration.append(round(time.time() * 1000 - start, 4))

    duration.append(plugin_duration)

### Duration(ms) results

In [ ]:
display(
    HTML(
        tabulate.tabulate(duration, headers=["Plugin", "Duration(ms)"], tablefmt="html")
    )
)

### Prediction performance

__Steps__
 - We train each prediction plugin on the dataset.
 - We report the Brier score and the C-Index metrics using cross validation.

In [ ]:
# autoprognosis absolute
from autoprognosis.utils.tester import evaluate_survival_estimator


def get_metrics(plugin):
    score = evaluate_survival_estimator(plugin, X, T, E, time_horizons)["str"]

    return score["c_index"], score["brier_score"]


plugins = predictors.list_available()

metrics_headers = ["Plugin", "C-INDEX", "Brier Score"]
test_score = []


for plugin in plugins:
    fproc = predictors.get(plugin)

    cindex, brier_score = get_metrics(fproc)

    test_score.append([plugin, cindex, brier_score])

In [ ]:
display(HTML(tabulate.tabulate(test_score, headers=metrics_headers, tablefmt="html")))

# Congratulations!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement towards Machine learning and AI for medicine, you can do so in the following ways!

### Star AutoPrognosis on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the tools we're building.

- [Star AutoPrognosis](https://github.com/vanderschaarlab/autoprognosis)
- [Star HyperImpute](https://github.com/vanderschaarlab/hyperimpute)
